In [ ]:
# A*
# Always finds optimal path
# Slower, requires more memory
# f(n)=g(n)+h(n)

# Heuristic Function (Its implementation is under heuristic_algo.ipynb)
# 1. Manhattan Distance: Only straight moves allowed
# 2. Euclidean Distance: If cost is truly based on distance (e.g., maps)
# 3. Diagonal Distance: If diagonal moves are allowed, Pathfinding in games (e.g., Chess, Grid-based AI, Mazes)


# 1. Manhattan Distance Heuristic
# Total cost function = Cost function + Manhattan Distance Heuristic
# f(n) = g(n) + h(n))
# Where, 
# a. (g(n)) = Cost Function 
# The cost function represents the actual number of moves taken to reach a state from the initial state.
# Here, cost = g(n), which is simply incremented by 1 for each valid move.
# b. (h(n)) = Manhattan Distance Heuristic 
# Manhattan distance calculates the sum of the absolute differences between the current position (i, j) 
# of a tile and its goal position (goal_x, goal_y).
# Formula:
# h(n) = E(|X_current - X_goal| + |Y_current - Y_goal|)


# 2. Euclidean Distance Heuristic
# (h(n)) = sqrt ( (current_cell.x – goal.x)2 + (current_cell.y – goal.y)2 )


# 3. Diagonal Distance Heuristic
# dx = abs(current_cell.x – goal.x)
# dy = abs(current_cell.y – goal.y)
# h = D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)
# where D is length of each node(usually = 1) and D2 is diagonal distance between each node (usually = sqrt(2) ). 

In [ ]:
# Now you can import the desired function or class
import import_ipynb
import heapq
import math
import eight_puzzle_node as EightPuzzle
import heuristic_algo as IDistance

In [7]:
# Cost function (g(n)) - Move count
class Cost:
    __path_length: int = 0
    
    def __init__(self):
        self.value = 0
    
    def update(self):
        """Cost function: Number of moves taken to reach the current state."""
        self.__path_length = self.__path_length + 1
    
    def get(self):
        return self.__path_length


In [ ]:
# Create A*
class AstarSearch:
    puzzle: EightPuzzle.EightPuzzle = None
    cost: Cost = None
    distance: IDistance.IDistance = None
    
    def __init__(self, puzzle: EightPuzzle.EightPuzzle, cost: Cost, distance: IDistance.IDistance):
        self.puzzle = puzzle
        self.cost = cost
        self.distance = distance
        print("AstarSearch")
    
    # def __init__(self, puzzle: EightPuzzle.EightPuzzle):
    #     self.puzzle = puzzle
    
    def solve(self):
        visited = set()
        priority_queue = [] 
        
        g_n = self.cost.get()  # Initial cost (0 moves)
        h_n = self.distance.calculate(self.puzzle)  # Initial heuristic
        f_n = g_n + h_n
        start_node = (f_n, g_n, self.puzzle, [])  # (f(n), g(n), puzzle, path)
        heapq.heappush(priority_queue, start_node)
    
        while priority_queue:
            _, g_n, current, path = heapq.heappop(priority_queue)
            state_tuple = tuple(map(tuple, current.state))
            
            if state_tuple in visited:
                continue
            
            visited.add(state_tuple)
            current.display_state()
            print("---------------")
            
            if current.is_goal_reached():
                print("Goal state reached!")
                print("Solution path:", path)
                return
            
            
            lists = current.get_possible_moves() 
            print(lists)
            for move in lists:
                next_state: EightPuzzle.EightPuzzle = current.move(move)
                if next_state:
                    self.cost.update()  # Increase cost g(n)
                    new_g_n = self.cost.get()
                    new_h_n = self.distance.calculate(next_state)
                    new_f_n = new_g_n + new_h_n
                    next_node = (new_f_n, new_g_n, next_state, path + [move])
                    heapq.heappush(priority_queue, next_node)
        
        print("No solution found")
        return None